In [1]:
import pandas as pd
import time
import numpy
import numpy as np
import json
from tqdm import tqdm
from numpy import linalg as LA
from scipy.sparse import csr_matrix

def mergeExpertise(dict1, dict2):
    dict3 = {**dict1, **dict2}
    for key, value in dict3.items():
        if key in dict1 and key in dict2:
               dict3[key] = value + dict1[key]
    return dict3


class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
    
def expertise_paper_update(author_expertise2update,author_expertise_in_paper,author_id_li):
    i = 0
    for author in author_id_li:
        if author in author_expertise2update.keys():
            author_expertise2update[author] += author_expertise_in_paper[i]
        else:
             author_expertise2update[author] = author_expertise_in_paper[i]
        i+= 1

def get_M_update(df_T,reverse_mesh_dic,mesh_dic):       

    df_all = df_T
    
    df_PM_all = df_all[df_all[2]== 'PM'].copy()

    Paper1 =  df_PM_all[0]
    Paper1 = set(Paper1)
     
    df_AP_all = df_all[df_all[2]== 'AP'].copy()
    Author =  df_AP_all[0]
    Author  = set(Author)
    
    Paper2 = df_AP_all[1]
    Paper2 = set(Paper2)
    Paper = Paper1.union(Paper2)

    paper_dic = {}
    reverse_paper_dic = {}
    for index,value in enumerate(Paper):
        paper_dic[index] = value
        reverse_paper_dic[value] = index
    
    author_dic = {}
    reverse_author_dic ={}
    for index,value in enumerate(Author):
        author_dic[index] = value
        reverse_author_dic[value] = index
            
#  index conversion
#  AP will be pre and T seperately for M_experience and M_update
#  PM will be global one as nothing will change


    df_tmp_T = df_T.copy()
    df_AP_T = df_tmp_T[df_tmp_T[2]== 'AP'].copy()
    
    df_AP_T['paper_index'] = df_AP_T[1].astype(str).apply(lambda x: reverse_paper_dic[x])
    df_AP_T['author_index'] = df_AP_T[0].apply(lambda x: reverse_author_dic[x])

    
    del df_AP_T[0]
    del df_AP_T[1]

    total_papers = len(Paper)
    total_authors = len(Author)
    
    rows = list(df_AP_T['author_index'])
    cols = list(df_AP_T['paper_index'])
    
    data_lenth = len(df_AP_T)
    data = [1] * data_lenth
    
    M_update = csr_matrix((data, (rows, cols)), shape=(total_authors, total_papers))
    
    df_PM_all['paper_index'] = df_PM_all[0].astype(str).apply(lambda x: reverse_paper_dic[x])

    df_PM_all['mesh_index'] = df_PM_all[1].apply(lambda x: reverse_mesh_dic[x])

    rows = list(df_PM_all['mesh_index'])
    cols = list(df_PM_all['paper_index'])
    
    data_lenth = len(df_PM_all)
    total_mesh = len(mesh_dic)

    data = [1] * data_lenth

    M_mp = csr_matrix((data, (rows, cols)), shape=(total_mesh, total_papers))

    return M_update.astype(bool),M_mp.astype(bool),author_dic,paper_dic,reverse_author_dic,reverse_paper_dic

def SHS_Alloc(M_update,M_mp,author_dic,author_expertise):
    author_expertise2update = dict()
    
    M_pa = M_update.T.tocsr() 
    M_pm = M_mp.T.tocsr() 
    ind = 0
    
    for paper_id in set(M_update.indices):
        
#         if ind%5000 == 0:
#             print(str(ind) + 'papers processed')
        ind +=1    
        
        co_authors = M_pa[paper_id,:].indices 
        mesh_in_paper = M_pm[paper_id,:].indices
        author_expertise_in_paper = np.zeros((len(co_authors),118))
        author_id_li = []
        author_paper_li = []
        for author in co_authors:
            author_id = author_dic[author]
            if author_id in author_expertise.keys():
                author_paper = np.array(author_expertise[author_id])
                author_paper = np.sqrt(sum(author_paper*author_paper) + 1)
            else:
#                 author_expertise[author_id] = {}
                author_paper = 1.0
                
            author_paper_li.append(author_paper)
            author_id_li.append(author_id)
        
        author_paper_li = np.array(author_paper_li)
        
        # paper list of all authors
#         for (author_id,author_paper) in zip(author_id_li,author_paper_li):
#             author_expertise[author_id]['pc'] =  author_paper
        
        for mesh in mesh_in_paper:  
            author_exp_li = []          
            for author_id in author_id_li:                
                # find author exprience
                try:                    
                    author_exp = author_expertise[author_id][mesh]
                    #print('upadate author')
                except(KeyError):
                    author_exp = 0 
                # expertise of all co-authors about the focal mesh
                author_exp_li.append(author_exp)  


            author_exp_li = np.array(author_exp_li)    

            # here we may want to chose from L1 or L2 norm

            # norm2, second denominator
            all_exp = np.sqrt(sum(author_exp_li*author_exp_li) + 1.0)

            author_update_li = (author_exp_li*author_exp_li + 1.0)/(author_paper_li * all_exp)
            
            # author_exp_li = author_exp_li + author_update_li
            author_expertise_in_paper[:,mesh] += author_update_li
            
        expertise_paper_update(author_expertise2update,author_expertise_in_paper,author_id_li)
            
    return author_expertise2update

def reduce_expertise(df_ae,author_li):
    # from dict to array
    df = df_ae

    all_mesh = set(mesh_nodes[0].values)

    exist_mesh = set(df.columns)

    none_col = all_mesh - exist_mesh

    none_col

    for name in none_col:
        df[name] = np.nan

    df = df.reindex(sorted(df.columns), axis=1)

    df = df.fillna(0)

    df = df.reindex(author_li)

    df = df.dropna()

    author_expertise_reduced = df.T.to_dict('list')
    
    return author_expertise_reduced


In [2]:
# change this to the file path on your side
file = open('author_expertise_'+str(1955) + '.json', 'r') 
author_expertise = json.load(file) 
file.close()

file = open('simple_expertise_'+str(1955) + '.json', 'r') 
simple_expertise = json.load(file) 
file.close()

mesh_nodes = pd.read_csv('mesh_node_list.csv',header = None)
mesh_dic = mesh_nodes[0].to_dict()
reverse_mesh_dic = {val: key for key, val in mesh_dic.items()}

In [3]:
# expertise generated by BL model
df_se = pd.DataFrame(simple_expertise)

df_se = df_se.T

# expertise generated by DHA model
df_ae = pd.DataFrame(author_expertise)

df_ae = df_ae.T

# delete paper_count as we are not using it here
del df_ae['pc']

df_ae.columns = df_ae.columns.astype(int).map(mesh_dic)

In [4]:
df_ae = df_ae.fillna(0)
df_se = df_se.fillna(0)

df_se_norm = df_se.div(df_se.sum(axis=1), axis=0)

df_ae_norm = df_ae.div(df_ae.sum(axis=1), axis=0)

load link list in the focusing year, this file contains link list of type "author-paper" and "paper-MeSH" at the same time, the last colomn indicates the type of link

In [5]:
year = 1956

df_link_T = pd.read_csv(r'link_list_1956.csv',header = None,dtype = str)

Example in Appendix 2: pmid == '13327374'

In [6]:
df_link_T = df_link_T[(df_link_T[1] == '13327374')|(df_link_T[0] == '13327374')]

In [7]:
paper_mesh = list(df_link_T[df_link_T[0] == '13327374'][1])

In [8]:
df_link_T

,0,1,2
120357,18121725_3,13327374,AP
120358,13327374_5,13327374,AP
120359,18898073_2,13327374,AP
120360,14803545_1,13327374,AP
120361,18898062_1,13327374,AP
176659,13327374,D06,PM
176660,13327374,D27,PM
237029,13327374,B01,PM
297295,13327374,D01,PM
410901,13327374,D23,PM


convert, reorder columns, reduce raws

In [9]:

df = df_ae

all_mesh = set(mesh_nodes[0].values)

exist_mesh = set(df.columns)

none_col = all_mesh - exist_mesh

none_col

for name in none_col:
    df[name] = np.nan

df = df.reindex(sorted(df.columns), axis=1)

df = df.fillna(0)

author_li = ['18121725_3', '13327374_5', '18898073_2', '14803545_1', '18898062_1']

df = df.reindex(author_li)

df = df.dropna()

author_expertise_reduced = df.T.to_dict('list')

In [10]:
author_li = ['18121725_3', '13327374_5', '18898073_2', '14803545_1', '18898062_1']
author_expertise_reduced = reduce_expertise(df_ae,author_li)

In [11]:
author_expertise_reduced.keys()

dict_keys(['18121725_3', '18898073_2', '14803545_1', '18898062_1'])

Apply the method and organize the result

In [12]:
M_u,Mmp,author_dic,paper_dic,reverse_author_dic,reverse_paper_dic = get_M_update(df_link_T,reverse_mesh_dic,mesh_dic)

In [13]:
author_expertise2update = SHS_Alloc(M_u,Mmp,author_dic,author_expertise_reduced)

In [14]:
res = pd.DataFrame(author_expertise2update).T

In [15]:
res.columns = df.columns

result containing all MeSH categories, which is not easy to read

In [16]:
res

,A01,A02,A03,A04,A05,A06,A07,A08,A09,A10,...,N02,N03,N04,N05,N06,V01,V02,V03,V04,Z01
18121725_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13327374_5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14803545_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18898073_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18898062_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


use the list of MeSH terms linked to this paper to reduce the result

In [17]:
paper_mesh

['D06',
 'D27',
 'B01',
 'D01',
 'D23',
 'A12',
 'C05',
 'C10',
 'C19',
 'G02',
 'G03',
 'G07',
 'C18']

this is the result in Table 9

In [18]:
res[paper_mesh]

,D06,D27,B01,D01,D23,A12,C05,C10,C19,G02,G03,G07,C18
18121725_3,0.184644,0.097456,0.083584,0.141158,0.041251,0.047520,0.005491,0.006378,0.092207,0.038189,0.027488,0.051060,0.038099
13327374_5,0.100653,0.183259,0.010679,0.165264,0.534818,0.347003,0.023264,0.081957,0.143765,0.424212,0.366029,0.339241,0.263124
14803545_1,0.140170,0.161840,0.002999,0.314881,0.110391,0.265421,0.003294,0.010501,0.033098,0.050171,0.072150,0.040809,0.157308
18898073_2,0.193327,0.113178,0.205994,0.066385,0.052928,0.025331,0.020299,0.006461,0.210856,0.083165,0.091864,0.095738,0.087153
18898062_1,0.011771,0.002352,0.761659,0.001699,0.004865,0.003115,0.371434,0.105972,0.001451,0.003884,0.003206,0.003024,0.003423


In [19]:
res[paper_mesh].reindex(author_li).round(3).fillna(0).to_latex('exp_added.tex')

In [20]:
df_ae.reindex(author_li)[paper_mesh].round(3).fillna(0).to_latex('author_exp.tex')

Author list:
14803545_1: HIOCO D  D01 A12
18898062_1: DE SEZE S B01 C05
13327374_5: GILLE H D23 G02 G03 A12 G07 learning
18121725_3: DELAVILLE M D06 D01	
18898073_2: RICHTWITZ A  B01 C19